Script to -
* Define the genmore tiles /regions before processing it in the enformer.
* Filter out for relevant chromosomes only
* Add starting extra tile window for the regions not covered.
* Remove duplicates in the regions.


In [1]:
import csv
import itertools

In [2]:
out_dir = 'data/'
!mkdir {out_dir}
window_size = 196608 
step_size_int = 114688
coverage = 1

mkdir: data/: File exists


### Get the chromosomes sizes

In [3]:
!wget https://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/hg38.chrom.sizes -O {out_dir}hg38.chrom.sizes

--2024-06-26 13:27:22--  https://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/hg38.chrom.sizes
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11672 (11K)
Saving to: ‘data/hg38.chrom.sizes’

data/hg38.chrom.siz 100%[===================>]  11.40K  --.-KB/s    in 0s      

2024-06-26 13:27:23 (27.9 MB/s) - ‘data/hg38.chrom.sizes’ saved [11672/11672]



### BED TOOLS for tiling

In [4]:
!bedtools makewindows -g {out_dir}hg38.chrom.sizes -w {window_size} -s {step_size_int} > {out_dir}hg38.{window_size}-{coverage}X.windows.bed

In [5]:
!wc -l {out_dir}hg38.{window_size}-{coverage}X.windows.bed
!head {out_dir}hg38.{window_size}-{coverage}X.windows.bed

   28249 data/hg38.196608-1X.windows.bed
chr1	0	196608
chr1	114688	311296
chr1	229376	425984
chr1	344064	540672
chr1	458752	655360
chr1	573440	770048
chr1	688128	884736
chr1	802816	999424
chr1	917504	1114112
chr1	1032192	1228800


In [6]:
print(28249 / (10**6), 'million regions')

0.028249 million regions


### Filter out the chromosomes

In [7]:
# filter to keep chromosome level assemblies
keep_chrs = ['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 
             'chr2', 'chr20', 'chr21', 'chr22', 
             'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chrM', 'chrX', 'chrY']

# Adding to a text file.
chrs_file = open(f"{out_dir}allowed_chroms.txt", 'w')
[chrs_file.write(chr_+'\n') for chr_ in keep_chrs]
chrs_file.close()
# !cat {out_dir}allowed_chroms.txt

In [8]:
!grep -Fwf {out_dir}allowed_chroms.txt {out_dir}hg38.{window_size}-{coverage}X.windows.bed > {out_dir}hg38.{window_size}-{coverage}X.windows.fixed.bed

In [9]:
!wc -l {out_dir}hg38.{window_size}-{coverage}X.windows.fixed.bed

   26940 data/hg38.196608-1X.windows.fixed.bed


In [10]:
awk_str = "awk -F'\t' '{print $1}'"
!{awk_str} {out_dir}hg38.{window_size}-{coverage}X.windows.fixed.bed | sort | uniq

chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr2
chr20
chr21
chr22
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chrM
chrX
chrY


### Manual adjustment for enformer input requirements

In [11]:
# add manual regions for the start of the genome which would be missed by enformer and adjust end regions
enformer_input_seq_length = 196608
enformer_embed_length     = 114688
enformer_add_pad          = 40960
enformer_one_sided = enformer_embed_length + enformer_add_pad

regions = []

# 1 - add extra start regions
for c in keep_chrs:
    if c != 'chrM': 
        regions.append([c,0,enformer_embed_length + enformer_add_pad])

# 2 - adjust the tail ends
with open(f'{out_dir}hg38.{window_size}-{coverage}X.windows.fixed.bed')as f:
    
    for line in f:
        line_data = line.strip().split()
        
        if (line_data[0] != 'chrM') and (int(line_data[2]) - int(line_data[1]) < enformer_input_seq_length):
            
            print(f"Am Short {line_data}")
            chr_start   = int(line_data[2]) - (enformer_embed_length + enformer_add_pad)
            chr_start_2 = int(line_data[2]) - (enformer_embed_length + enformer_add_pad + enformer_add_pad)
            regions.append([line_data[0],int(chr_start_2),int(line_data[2])])
        else:
            chr_start = line_data[1]
            
        regions.append([line_data[0],int(chr_start),int(line_data[2])])

# 3 - remove duplicates adjusting tail ends generates duplicates
regions.sort()
regions = list(regions for regions,_ in itertools.groupby(regions))

# 4 - save it back in the bed file
with open(f'{out_dir}hg38.{window_size}-{coverage}X.windows.fixed1.bed', 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(regions)
    
!wc -l {out_dir}hg38.{window_size}-{coverage}X.windows.fixed1.bed

Am Short ['chr1', '248872960', '248956422']
Am Short ['chr2', '242106368', '242193529']
Am Short ['chr3', '198180864', '198295559']
Am Short ['chr3', '198295552', '198295559']
Am Short ['chr4', '190152704', '190214555']
Am Short ['chr5', '181436416', '181538259']
Am Short ['chr6', '170655744', '170805979']
Am Short ['chr6', '170770432', '170805979']
Am Short ['chr7', '159301632', '159345973']
Am Short ['chrX', '155975680', '156040895']
Am Short ['chr8', '145080320', '145138636']
Am Short ['chr9', '138313728', '138394717']
Am Short ['chr11', '134987776', '135086622']
Am Short ['chr10', '133726208', '133797422']
Am Short ['chr12', '133152768', '133275309']
Am Short ['chr12', '133267456', '133275309']
Am Short ['chr13', '114229248', '114364328']
Am Short ['chr13', '114343936', '114364328']
Am Short ['chr14', '106889216', '107043718']
Am Short ['chr14', '107003904', '107043718']
Am Short ['chr15', '101842944', '101991189']
Am Short ['chr15', '101957632', '101991189']
Am Short ['chr16', '90